In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import proyeccion_rdr.features as features
import proyeccion_rdr.produccion.a03_pabellon as pb

pd.set_option("display.max_columns", None)


ANIO_INICIO = 2017
ANIO_TERMINO = 2035
# Agrega las columnas de poblacion de interes
ANIOS_INTERES = [str(i) for i in range(2017, 2036)]

2025-05-08 13:55:31.767 | INFO     | proyeccion_rdr.config:<module>:11 - PROJ_ROOT path is: C:\Users\jarojas\Desktop\Javier\SSMN\EPH - RDR\2.0-proyeccion-rdr


## Obtencion de casos por area de influencia INT

In [3]:
# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
_, casos_macroproceso = features.leer_casos_macroprocesos(RUTA_ARCHIVOS)

# Obtiene los casos quirurgicos
casos_quirurgicos = casos_macroproceso.query("tipo_paciente == 'quirurgicos'")

In [4]:
# Los diagnosticos que tienen un porcentaje de hosp. mayor a 0
diagnosticos_relevantes = list(casos_quirurgicos.index)

# Ademas, los diagnosticos que estan agrupados dentro de los que aportan a AC
# Solo deja los diagnosticos que esten agrupados
diagnosticos_a_reasignar = casos_quirurgicos.dropna(subset="Diagnosticos Contenidos")

# Separa la lista de diagnosticos en filas
diagnosticos_agrupados = diagnosticos_a_reasignar.explode("Diagnosticos Contenidos")
diagnosticos_agrupados = list(diagnosticos_agrupados["Diagnosticos Contenidos"].unique())

# Suma los diagnosticos relevantes y los agrupados
diagnosticos_a_buscar_en_quir = sorted(diagnosticos_relevantes + diagnosticos_agrupados)

## Obtencion de tiempos quirurgicos entre 2015 y 2022 en pabellon para diagnosticos relevantes

Estos se obtendran a partir de la union de la base de GRD (2015 a 2022, tiene los diagnosticos codificados) y Pabellon (2015 a 2022, tiene los tiempos quirurgicos).

In [5]:
# Lee la base de GRD unida con Pabellon
grd_y_pabellon = pd.read_csv("../data/raw/5_grd_y_pabellon/df_procesada_grd_y_pabellon.csv")
grd_y_pabellon["t_total"] = pd.to_timedelta(grd_y_pabellon["t_total"].str.split(" ").str[1])

C:\Users\jarojas\AppData\Local\Temp\ipykernel_25860\3808083235.py:2: DtypeWarning: Columns (41,42,43,44,45,46,47,48,49,50,51,52,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,149,150,151,152,153,154,155,156,157,158,230) have mixed types. Specify dtype option on import or set low_memory=False.
  grd_y_pabellon = pd.read_csv("../data/raw/5_grd_y_pabellon/df_procesada_grd_y_pabellon.csv")


In [6]:
# Filtra solamente los diagnosticos que aportan a quirurgico
grd_y_pabellon_relevantes = grd_y_pabellon.query(
    "diagnostico_1.isin(@diagnosticos_a_buscar_en_quir)"
).copy()

## Reasginar diagnosticos

In [7]:
for row in diagnosticos_a_reasignar.itertuples():
    diagnostico_nuevo = row[0]
    diagnosticos_antiguos = row[1]

    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    diagnosticos_cambiados = grd_y_pabellon_relevantes["diagnostico_1"].replace(
        diagnosticos_antiguos, diagnostico_nuevo
    )
    grd_y_pabellon_relevantes["diagnostico_1"] = diagnosticos_cambiados

Cambiando ['Q262', 'K914', 'K638'] a ENTN
Cambiando ['H652', 'H653', 'H654', 'H669'] a HOTI
Cambiando ['J121', 'J122', 'J123', 'J129', 'J159', 'J189'] a JNEU
Cambiando ['K352', 'K353', 'K358'] a KAPE
Cambiando ['K565', 'K650'] a KGAST
Cambiando ['M411', 'M410', 'M412', 'M414', 'T813', 'Q763'] a MESC
Cambiando ['K560', 'Q625', 'T222'] a OBSI
Cambiando ['K36X', 'K659'] a PERC
Cambiando ['Q390', 'Q391', 'Q393'] a QATR
Cambiando ['Q375', 'Q374', 'Q369', 'Q371', 'Q379', 'Q373', 'Q370', 'T813', 'Q355', 'Q372', 'Q359', 'Q378'] a QLAB
Cambiando ['S424', 'S524', 'S525', 'S528', 'S529'] a SFRA
Cambiando ['T213', 'T202', 'T212', 'T203', 'T293', 'T201', 'T273', 'T223', 'T222', 'T243', 'T280', 'T211', 'T242', 'T253', 'T232', 'T221', 'T241', 'T312', 'T291', 'T292', 'T252'] a TQUE


## Obtener tiempos quirurgicos

In [12]:
# Obtiene el resumen de duraciones de las int. q por diagnostico acumuladas
tiempos_quir = pd.to_timedelta(
    grd_y_pabellon_relevantes.groupby(["diagnostico_1"])["t_total"].describe()["75%"]
)

In [13]:
# Calcular tiempo utilizado en pabellón en horas
tiempo_utilizado_pabellon_horas = pb.calcular_tiempo_utilizado_pabellon(
    casos_quirurgicos[ANIOS_INTERES],
    tiempos_quir,
)

# Calcular horas laborales
horas_laborales = features.calcular_horas_laborales(ANIO_INICIO, ANIO_TERMINO, 12)

# Calcular cantidad de pabellones necesarios
cantidad_de_pabellones_necesarios = pb.calcular_cantidad_de_pabellones_necesarios(
    tiempo_utilizado_pabellon_horas, horas_laborales
)

# Obtiene la suma total de pabellones
suma_total_pabellones = cantidad_de_pabellones_necesarios.sum()

Tiempo utilizado en pabellón calculado (en horas):
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| Diagnostico |        2017        |        2018        |        2019        |        2020        |        2021        |        2022        |        2023        |        2024        |        2025        |        2026        |        2027        |        2028        |        2029        |        2030        |        2031        |        2032        |        2033        |        2034        |        2035        |
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------

In [14]:
print(f"> Pabellones 2035: {suma_total_pabellones['2035']:.2f}")

> Pabellones 2035: 2.35


## Complicaciones

In [15]:
# # Define las complicaciones a buscar
# a_buscar_operaciones = {
#     "ecmo": r"ECMO",
#     "trasplantes": r"TRASPLANTE|TRANSPLANTE",
#     "aseos": r"ASEO",
#     "drenajes": r"DRENAJE",
#     "traqueostomias": r"TRAQUEOSTOMIA|TRAQUEOSTOMÍA",
#     "reintervenciones_reoperaciones": r"REINTERVENCION|REOPERACION|REINTERVENCIÓN|REOPERACIÓN",
# }

# # Define los diagnosticos que son complicaciones
# a_buscar_diagnosticos = {
#     "empiemas": r"EMPIEMA",
#     "rupturas": r"ANEURISMA",
# }

# # Busca los nombres de las operaciones en la base de pabellon
# resultados_operaciones = pb.iterar_en_complicaciones_a_buscar(
#     grd_y_pabellon, a_buscar_operaciones, "intervencion_quirurgica"
# )

# # Busca los diagnosticos en el primer diagnostico y segundo
# resultados_diagnosticos = pb.iterar_en_complicaciones_a_buscar(
#     grd_y_pabellon, a_buscar_diagnosticos, "diagnostico"
# )

# # Obtiene resumen de complicaciones
# resumen_complicaciones = pd.concat([resultados_operaciones, resultados_diagnosticos])

In [16]:
# Obtiene los casos por especialidad por anio
casos_especialidad_desglosado = casos_macroproceso.query("tipo_paciente == 'Placeholder'")
casos_especialidad = casos_especialidad_desglosado.groupby("tipo_paciente")[ANIOS_INTERES].sum()

In [17]:
# # Une los datos de complicaciones y los casos de especialidad
# casos_complicaciones = (
#     casos_especialidad.merge(
#         resumen_complicaciones[["fraccion", "tiempo_operacion_75%", "complicacion"]],
#         how="left",
#         left_index=True,
#         right_index=True,
#     )
#     .reset_index()
#     .sort_values(["complicacion", "tipo_paciente"])
#     .set_index(["complicacion", "tipo_paciente"])
# )

# # Agrega tiempos de limpieza de pabellones
# casos_complicaciones["tiempo_operacion_75%"] = casos_complicaciones[
#     "tiempo_operacion_75%"
# ] + pd.Timedelta("30 minutes")

# # Obtiene los casos de complicaciones
# casos_complicaciones[ANIOS_INTERES] = casos_complicaciones[ANIOS_INTERES].mul(
#     casos_complicaciones["fraccion"], axis=0
# )

# # Indica la cantidad de trasplantes
# CASOS_TRASPLANTES = 38
# casos_complicaciones.loc[("trasplantes", "CV"), ANIOS_INTERES] = CASOS_TRASPLANTES

# # Obtiene el tiempo de pabellon por las complicaciones
# tiempo_utilizado_pabellon_complicaciones_horas = (
#     casos_complicaciones[ANIOS_INTERES]
#     .mul(casos_complicaciones["tiempo_operacion_75%"], axis=0)
#     .apply(lambda x: x.dt.total_seconds() / 3600)
# )

# # Indica las horas para pabellon de emergencia
# horas_pabellon_emergencia = {str(anio): 365.25 * 24 for anio in range(ANIO_INICIO, ANIO_TERMINO + 1)}
# horas_pabellon_emergencia = pd.Series(horas_pabellon_emergencia)

# # Divide por las horas laborales
# pabellones_por_complicaciones = pb.calcular_cantidad_de_pabellones_necesarios(
#     tiempo_utilizado_pabellon_complicaciones_horas, horas_pabellon_emergencia
# )

# # # Obtiene los pabellones de emergencia por anio
# suma_total_pabellones_complicaciones = pabellones_por_complicaciones.sum()

In [18]:
# print(f"> Pabellones de Emergencia 2035: {suma_total_pabellones_complicaciones['2035']:.2f}")

In [19]:
# Carga el resumen de area de estudio para MINSAL
resumen_area_de_estudio_minsal = pd.read_excel(
    "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_RDR"
).sort_values("Diagnostico")
resumen_area_de_estudio_minsal["diagnostico_separado"] = (
    resumen_area_de_estudio_minsal["Diagnostico"].str.split(" - ").str[0]
)
resumen_area_de_estudio_minsal = resumen_area_de_estudio_minsal.set_index("diagnostico_separado")

# Sintetiza informacion de pabellon para el 2035
resumen_quirurgicos = pd.DataFrame(
    {
        "especialidad_quirurgica": casos_especialidad_desglosado["tipo_paciente"],
        "casos_quirurgicos_2035": casos_quirurgicos["2035"],
        "tiempo_quirurgico_75%": tiempos_quir,
        "horas_pabellon_2035": tiempo_utilizado_pabellon_horas["2035"],
        "horas_laborales_2035_pabellon_12_hrs": horas_laborales["2035"],
        "cantidad_de_pabellones": cantidad_de_pabellones_necesarios["2035"],
    }
)

# Une ambos resumenes segun el diagnostico
resumen_minsal = resumen_area_de_estudio_minsal.merge(
    resumen_quirurgicos, how="inner", left_index=True, right_index=True
)

In [20]:
a_guardar = {
    "resumen_MINSAL": resumen_minsal,
    "casos_quirurgicos": casos_especialidad_desglosado,
    "casos_por_especialidad": casos_especialidad,
    # "casos_complicaciones": casos_complicaciones,
    "resumen_duraciones_int_q_rel": tiempos_quir,
    "tiempo_utilizado_pabellon": tiempo_utilizado_pabellon_horas,
    "horas_laborales_por_anio": horas_laborales,
    "pabellones_desg": cantidad_de_pabellones_necesarios,
    "pabellones": suma_total_pabellones,
    # "tiempo_ut_emergencia": tiempo_utilizado_pabellon_complicaciones_horas,
    # "pabellones_desg_emergencia": pabellones_por_complicaciones,
    # "pabellones_emergencia": suma_total_pabellones_complicaciones,
}

In [21]:
with pd.ExcelWriter("../data/interim/2.0_estimacion_pabellones_RDR.xlsx") as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)